Note: This was an attempt at matching natural persons with their OC records. The problem is that the ones who hold a share in a company are not necessarily on the board of any company, so the matching is not very reliable; what would help a lot is to get the date of birth of each person mentioned in the BaFin data.

In [1]:
import pandas as pd
import os
import dotenv
import requests
import json
from tqdm import tqdm

# set up environment variables
dotenv.load_dotenv()
OPENCORPORATES_API_KEY = os.getenv("OPENCORPORATES_API_KEY")

In [12]:
df = pd.read_csv('../data/combined.csv')

In [ ]:
df.head()

In [14]:
# how many rows do not have a value for OpenCorporates URL?
print(df['OpenCorporates URL'].isna().sum())

7003


In [11]:
# Set up the OpenCorporates API URL
api_base_url = "https://api.opencorporates.com/v0.4/officers/search?q={}&api_token={}"

# Initialize a counter to keep track of processed rows
processed_rows = 0

# Iterate through the DataFrame
for row_index, row in df.iterrows():
    officer_name = row['notifying_party']

    # Skip rows with non-empty OpenCorporates URL
    if not pd.isna(row['OpenCorporates URL']):
        continue

    # Construct the OpenCorporates API URL using the officer name and API key
    api_url = api_base_url.format(officer_name, OPENCORPORATES_API_KEY)

    response = requests.get(api_url)

    if response.status_code == 200:
        data = json.loads(response.text)

    print(data)

    #     # Retrieve the officer ID from the JSON response and add it to the OpenCorporates URL column
    #     try:
    #         df.at[row_index, 'opencorporates_url'] = data['officers'][0]['officer']['id']
    #     except (KeyError, IndexError):
    #         df.at[row_index, 'opencorporates_url'] = 'no match'
    # else:
    #     print(f"API request failed for row {row_index}: {response.status_code}")

    processed_rows += 1

    if processed_rows >= 20:
        # Save the DataFrame to a CSV file after processing the first 20 rows
        df.to_csv('../data/officer_data_added.csv', index=False)
        break

{'api_version': '0.4', 'results': {'page': 1, 'per_page': 30, 'total_pages': 0, 'total_count': 0, 'officers': []}}
{'api_version': '0.4', 'results': {'page': 1, 'per_page': 30, 'total_pages': 0, 'total_count': 0, 'officers': []}}
{'api_version': '0.4', 'results': {'page': 1, 'per_page': 30, 'total_pages': 1, 'total_count': 1, 'officers': [{'officer': {'id': 311322663, 'uid': None, 'name': 'Franz Jürgen Schneider', 'jurisdiction_code': 'de', 'position': 'persönlich haftender gesellschafter', 'retrieved_at': '2018-08-24T18:01:02+00:00', 'opencorporates_url': 'https://opencorporates.com/officers/311322663', 'start_date': '2008-09-02', 'end_date': None, 'occupation': None, 'current_status': None, 'inactive': True, 'address': None, 'nationality': None, 'date_of_birth': '1943-09-15', 'company': {'name': 'Gut Thurbruch OHG', 'jurisdiction_code': 'de', 'company_number': 'N1209V_HRA2187', 'opencorporates_url': 'https://opencorporates.com/companies/de/N1209V_HRA2187'}}}]}}
{'api_version': '0.4',

KeyboardInterrupt: 

In [6]:
import pprint

api_base_url = "https://api.opencorporates.com/v0.4/officers/search?q={}&api_token={}"

officer_name = "Schneider, Franz Jürgen"
api_url = api_base_url.format(officer_name, OPENCORPORATES_API_KEY)

response = requests.get(api_url)
data = json.loads(response.text)

# pprint data
pprint.pprint(data)

{'api_version': '0.4',
 'results': {'officers': [{'officer': {'address': None,
                                       'company': {'company_number': 'N1209V_HRA2187',
                                                   'jurisdiction_code': 'de',
                                                   'name': 'Gut Thurbruch OHG',
                                                   'opencorporates_url': 'https://opencorporates.com/companies/de/N1209V_HRA2187'},
                                       'current_status': None,
                                       'date_of_birth': '1943-09-15',
                                       'end_date': None,
                                       'id': 311322663,
                                       'inactive': True,
                                       'jurisdiction_code': 'de',
                                       'name': 'Franz Jürgen Schneider',
                                       'nationality': None,
                                       'oc